In [1]:
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import json
import pdb
import glob
from nilearn import plotting as nip
from nilearn import image
import pandas as pd

Finds all the PyNets atlases. Want to remove file extension since this will be used to load the Nifti file and write to the JSON file with the same name.

In [2]:
mypath = str(Path().absolute())
labelpath = join(mypath,'PyNets/Labels')
label_files = [f for f in os.listdir(labelpath) if os.path.isfile(join(labelpath,f))]
files = [f.split('.')[0] for f in label_files]
files = [i for i in files if not i.startswith("__")]
files = [i for i in files if not i.startswith("aal_c")]
files = [i for i in files if not i.startswith("Random")]
files = [i for i in files if not i.startswith("Voxel")]
files = files[5:len(files)-1]
files

['DesikanKlein2012',
 'AALTzourioMazoyer2002',
 'AICHAreorderedJoliot2015',
 'AAL2zourioMazoyer2002',
 'AICHAJoliot2015',
 'MICCAI2012MultiAtlasLabelingWorkshopandChallengeNeuromorphometrics']

For each atlas, load in the metadata file currently in .txt format from PyNets. Then create a JSON file with this basic information: the labels for each region. Load in the Nifti file and calculate the size per region

In [104]:
for f in files:
    print(f)
    try:
        filename = str(f)+'.txt'
        filename = join(labelpath,filename)
        df = pd.read_csv(filename, sep=' ', header=None, usecols=[0,1])
        df.columns = ["Region", "Label"]
    except ValueError:
        filename = str(f)+'.txt'
        filename = join(labelpath,filename)
        df = pd.read_csv(filename, sep='\t', header=None, usecols=[0,1])
        df.columns = ["Region", "Label"]
    outname = f+'.json'
    jsonpath = join(mypath,'PyNets/JSONs')
    outname = join(jsonpath,outname)
    createJSON(df,outname)
    
    niftipath = join(mypath,'PyNets/atlases/')
    niftiname = f+'.nii.gz'
    niftiname = join(niftipath,niftiname)
    Size_JSON(outname, niftiname, outname)
    

AALTzourioMazoyer2002
<class 'dict'>
<class 'dict'>
AICHAreorderedJoliot2015
<class 'dict'>
<class 'dict'>
AAL2zourioMazoyer2002
<class 'dict'>
<class 'dict'>
AICHAJoliot2015
<class 'dict'>
<class 'dict'>
MICCAI2012MultiAtlasLabelingWorkshopandChallengeNeuromorphometrics
<class 'dict'>
<class 'dict'>


Creates a JSON file. df is a pandas dataframe with columns "Region" (the region number) and "Label" (the corresponding label for the region). Puts the data into JSON format.

In [94]:
def createJSON(df, outname):
    data = {}
    data['0'] = {}
    data['0']['label'] = 'empty'
    print(type(data))
    for row in range(1,len(df)+1):
        data[str(row)]={}
        data[str(row)]['label'] = df['Label'][row-1]
    with open(outname, 'w') as outfile:  
        json.dump(data, outfile, indent=4, sort_keys=True)

Calculates the size of each region in an atlas. json_file is the name of the current JSON_file. nifti_file is name of the Nifti_file. outname is the name of the JSON_file where the Size data should be added. The default for outname is the json_file. 

In [95]:
def Size_JSON(json_file, nifti_file, outname=json_file):    
    img = nib.load(nifti_file)
    img_data = img.get_fdata()
    img_flat = img_data.flatten()
    img_flat.tolist()
    img_flat = [int(i) for i in img_flat]
    values = np.unique(img_flat)
    counts = np.array(range(1, len(img_flat)+1))
    counts = [int(i) for i in counts]
    img_flatten = list(zip(img_flat, counts))
    img_flatten = sorted(img_flatten, key = lambda l:l[0])

    img_cell = [[] for i in range(len(values))]
    start = 0
    for i in range(0,len(values)):
        img_list = [x[0] for x in img_flatten]
        end = max(loc for loc, val in enumerate(img_list) if val == values[i])
        for j in range(start,end):
            img_cell[i].append(img_flatten[j][1])  
        start = end+1
    
    with open(json_file) as f:
        data = json.load(f)
        print(type(data))
    
    for i in range(0,len(values)):
        data[str(i)]["size"] = str(len(img_cell[i]))
        #val = str(len(img_cell[i]))
        #print(val)
        #data[str(i)].append({
        #    'size': str(len(img_cell[i]))
        #})
    
    with open(outname, 'w') as f:
        json.dump(data, f, indent=4, sort_keys=True)

Calculates the center of mass for each region in a Nifti file. brain is the nibabel object after loading the Nifti file.

In [1]:
def get_centers(brain, outfile):
    """
    Get coordinate centers given a nifti image loaded with nibabel
    
    Returns a dictionary of label: coordinate as an [x, y, z] array
    """
    dat = brain.get_data()
    dat = np.squeeze(dat)
    labs = np.unique(dat)
    labs = labs[labs != 0]
    fd_dat = np.stack([np.asarray(dat == lab).astype('float64') for lab in labs], axis=3)
    parcels = nib.Nifti1Image(dataobj=fd_dat, header=brain.header, affine=brain.affine)
    regions_imgs = image.iter_img(parcels)
    # compute the centers of mass for each ROI
    coords_connectome = [nip.find_xyz_cut_coords(img) for img in regions_imgs]

    
    return dict(zip(labs, coords_connectome))

For each JSON file, add the centers to each region

In [4]:
def main():
    for f in files:
        print(f)
        filename = f+'.json'
        jsonpath = join(mypath, 'PyNets/JSONs')
        outfile = join(jsonpath,filename)
        niftipath = join(mypath,'PyNets/atlases/')
        niftiname = f+'.nii.gz'
        niftiname = join(niftipath,niftiname)
        parcel_im = nib.load(niftiname)
        get_centers(parcel_im,outfile)
        parcel_centers = get_centers(parcel_im,outfile)
        
        with open(outfile) as js:
            js_contents = json.load(js)
            for (k, v) in js_contents.items():
                try:
                    js_contents[k]["center"] = parcel_centers[int(k)]
                except KeyError:
                    js_contents[k]["center"] = None
                    with open(outfile, 'w') as jso:
                        json.dump(js_contents, jso, indent=4)
    return parcel_centers

In [5]:
main()

DesikanKlein2012
here
here2
here
here2
AALTzourioMazoyer2002
here
here2
here
here2
AICHAreorderedJoliot2015
here
here2
here
here2
AAL2zourioMazoyer2002
here
here2
here
here2
AICHAJoliot2015
here
here2
here
here2
MICCAI2012MultiAtlasLabelingWorkshopandChallengeNeuromorphometrics
here
here2
here
here2


{4: [0.5618556701030997, -9.103092783505161, -4.762886597938149],
 11: [0.2473763118440786, -44.59745127436281, -32.6671664167916],
 23: [9.59302325581396, 12.688953488372078, -7.831395348837205],
 30: [-9.408376963350776, 11.607329842931932, -8.340314136125656],
 31: [22.098930481283418, -4.528074866310163, -20.68716577540107],
 32: [-21.860000000000014, -5.740000000000009, -20.200000000000003],
 35: [0.5877383015597957, -30.154029462738308, -31.853119584055456],
 36: [13.870356037151709, 10.279411764705856, 9.697755417956657],
 37: [-12.969047619047629, 9.289285714285711, 9.105952380952388],
 38: [25.730695266272193, -62.37352071005917, -37.28498520710059],
 39: [-25.068228404099557, -62.481405563689606, -37.35856515373352],
 40: [18.836187332370542, -51.60748916855786, -35.73695069114916],
 41: [-17.885105092966853, -51.99009700889248, -35.770210185933706],
 44: [27.69475592073465, -15.422740454325762, 17.191692846785884],
 45: [-26.808739936569893, -16.954263234935354, 17.204970724